In [9]:
!pip install imutils
import numpy as np
import pandas as pd
import os
import random
import cv2
import imutils
import random
from sklearn import svm
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import image
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Pre-Processing

In [10]:
dir = "../input/handwritten-characters/Train/"
train_data = []
img_size = 32
non_chars = ["#", "$", "&", "@"]
for i in os.listdir(dir):
    if i in non_chars:
        continue
    count = 0
    sub_directory = os.path.join(dir, i)
    for j in os.listdir(sub_directory):
        count += 1
        if count > 4000:
            break
        img = cv2.imread(os.path.join(sub_directory, j), 0)
        img = cv2.resize(img, (img_size, img_size))
        train_data.append([img, i])

In [33]:
val_dir = "../input/handwritten-characters/Validation/"
val_data = []
img_size = 32
for i in os.listdir(val_dir):
    if i in non_chars:
        continue
    count = 0
    sub_directory = os.path.join(val_dir, i)
    for j in os.listdir(sub_directory):
        count += 1
        if count > 1000:
            break
        img = cv2.imread(os.path.join(sub_directory, j), 0)
        img = cv2.resize(img, (img_size, img_size))
        val_data.append([img, i])

In [34]:
random.shuffle(train_data)
random.shuffle(val_data)

train_X = []
train_Y = []
for features, label in train_data:
    train_X.append(features)
    train_Y.append(label)

val_X = []
val_Y = []
for features, label in val_data:
    val_X.append(features)
    val_Y.append(label)

# Feature Extraction

In [35]:
# Extract features from images
n_samples = len(train_X)
n_features = img_size * img_size
train_X = np.array(train_X).reshape(n_samples, -1)

n_samples_val = len(val_X)
val_X = np.array(val_X).reshape(n_samples_val, -1)

# Scale the data
n_components = 64  # Adjust the number of components as needed
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True).fit(train_X)

train_X = pca.transform(train_X)
val_X = pca.transform(val_X)

# Training SVM model

In [ ]:
# SVM model
svm_model = svm.SVC(kernel='linear', C=1.0)

# Train the SVM
svm_model.fit(train_X, train_Y)

# Make predictions on the validation set
val_Y_pred = svm_model.predict(val_X)

# Evaluate the SVM model
accuracy = accuracy_score(val_Y, val_Y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(svm_model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

# Testing on characters

In [ ]:
import cv2
import numpy as np

# Load and preprocess the test image
test_image_path = '/kaggle/input/handwritten-characters/Validation/4/0.jpg'
img = cv2.imread(test_image_path, 0)  # Load the image in grayscale
plt.imshow(img)
img = cv2.resize(img, (img_size, img_size))  # Resize to the same dimensions as training data

# Flatten the image into a 1D array
img_flat = img.reshape(-1)

# If necessary, apply the same preprocessing as you did for training data, such as PCA transformation or normalization.
img_transformed = pca.transform([img_flat])  # If you applied PCA transformation

# Make predictions using the SVM model
predn = svm_model.predict(img_transformed)
print(predn)

In [ ]:
import cv2
import numpy as np

# Load and preprocess the test image
test_image_path = '/kaggle/input/handwritten-characters/Validation/2/0.jpg'
img = cv2.imread(test_image_path, 0)  # Load the image in grayscale

img = cv2.resize(img, (img_size, img_size))  # Resize to the same dimensions as training data
plt.imshow(img)
# Flatten the image into a 1D array
img_flat = img.reshape(-1)

# If necessary, apply the same preprocessing as you did for training data, such as PCA transformation or normalization.
img_transformed = pca.transform([img_flat])  # If you applied PCA transformation

# Make predictions using the SVM model
predn = svm_model.predict(img_transformed)
print(predn)

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

# Grapheme segmentation

In [ ]:
import cv2
import imutils
import numpy as np

def grapheme_segmentation(img):
    graphemes = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = gray[y:y + h, x:x + w]
            graphemes.append(roi)
    
    return graphemes, image

In [ ]:
def recognize_characters(graphemes, svm_model, pca, img_size):
    letters = []
    
    for roi in graphemes:
        thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (img_size, img_size), interpolation=cv2.INTER_CUBIC)
        img_flat = thresh.reshape(-1)
        thresh = pca.transform([img_flat])
        ypred = svm_model.predict(thresh)
        letters.append(ypred[0])
    
    return letters

In [ ]:
def get_word(letter):
    word = "".join(letter)
    return word

In [ ]:
img_size = 32

img_path = '/kaggle/input/handwriting-recognition/test_v2/test/TEST_0005.jpg'

graphemes, image = grapheme_segmentation(img_path)
letters = recognize_characters(graphemes, svm_model, pca, img_size)
word = get_word(letters)
print("Recognized Word:", word)
plt.imshow(image)

# K-Means Clustering

In [ ]:
from sklearn.neighbors import NearestNeighbors

n_clusters = 40

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(train_X)
nn_model = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(kmeans.cluster_centers_)


In [ ]:
correct_predictions = 0
total_samples = len(val_X)

for i in range(total_samples):
    nearest_cluster_index = nn_model.kneighbors(val_X, n_neighbors=1)[1][0][0]
    
    predicted_character = train_Y[np.where(kmeans.labels_ == nearest_cluster_index)[0][0]]
    
    if predicted_character == val_Y[i]:
        correct_predictions += 1
accuracy = (correct_predictions / total_samples) * 30
print(f"Accuracy using Nearest Neighbors with K-means: {accuracy*100:.2f}%")

# Phase-2

**Step1** : Build a digit(0-9) + A-Z characters classifier using a CNN architecture. 

**Step2**: Apply character segmentation for the handwritten word image.

**Step3**: Classify each segmented letter and then get the final word in the image.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from keras.utils import np_utils
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
LB = LabelBinarizer()
train_Y = LB.fit_transform(train_Y)
val_Y = LB.fit_transform(val_Y)

In [ ]:
print(train_X.shape,val_X.shape)
print(train_X.shape,val_X.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(32,32,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(35, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

In [ ]:
history = model.fit(train_X,train_Y, epochs=5, batch_size=32, validation_data = (val_X, val_Y),  verbose=1)